In [ ]:
!pip install opendatasets

In [ ]:
import opendatasets as od

In [ ]:
od.download("https://www.kaggle.com/datasets/tarkkaanko/amazon", force = True)

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: mithilkatkar
Your Kaggle Key: ··········


100%|██████████| 582k/582k [00:00<00:00, 88.4MB/s]

In [ ]:
!pip install fasttext

In [ ]:
import fasttext
import pandas as pd

In [ ]:
df = pd.read_csv("/content/amazon/amazon_reviews.csv")

In [ ]:
df.head()

,Unnamed: 0,reviewerName,overall,reviewText,reviewTime,day_diff,helpful_yes,helpful_no,total_vote,score_pos_neg_diff,score_average_rating,wilson_lower_bound
0,0,NaN,4.0,No issues.,2014-07-23,138,0,0,0,0,0.0,0.0
1,1,0mie,5.0,"Purchased this for my device, it worked as adv...",2013-10-25,409,0,0,0,0,0.0,0.0
2,2,1K3,4.0,it works as expected. I should have sprung for...,2012-12-23,715,0,0,0,0,0.0,0.0
3,3,1m2,5.0,This think has worked out great.Had a diff. br...,2013-11-21,382,0,0,0,0,0.0,0.0
4,4,2&amp;1/2Men,5.0,"Bought it with Retail Packaging, arrived legit...",2013-07-13,513,0,0,0,0,0.0,0.0


In [ ]:
columns_to_remove = ['Unnamed: 0', 'reviewerName','reviewTime','day_diff','helpful_yes','helpful_no',	'total_vote',	'score_pos_neg_diff','score_average_rating','wilson_lower_bound']

In [ ]:
df = df.drop(columns=columns_to_remove)

In [ ]:
df

,overall,reviewText
0,4.0,No issues.
1,5.0,"Purchased this for my device, it worked as adv..."
2,4.0,it works as expected. I should have sprung for...
3,5.0,This think has worked out great.Had a diff. br...
4,5.0,"Bought it with Retail Packaging, arrived legit..."
...,...,...
4910,1.0,I bought this Sandisk 16GB Class 10 to use wit...
4911,5.0,Used this for extending the capabilities of my...
4912,5.0,Great card that is very fast and reliable. It ...
4913,5.0,Good amount of space for the stuff I want to d...


In [ ]:
texts = df['reviewText'].tolist()
labels = df['overall'].tolist()

In [ ]:
# Create a train and validation set
train_size = int(0.9 * len(texts))
train_texts = texts[:train_size]
train_labels = labels[:train_size]

valid_texts = texts[train_size:]
valid_labels = labels[train_size:]

In [ ]:
# Save the training and validation sets to a file
with open('train.txt', 'w') as f:
    for text, label in zip(train_texts, train_labels):
        f.write(f'__label__{label} {text}\n')

with open('valid.txt', 'w') as f:
    for text, label in zip(valid_texts, valid_labels):
        f.write(f'__label__{label} {text}\n')


In [ ]:
# Train the fastText model
model = fasttext.train_supervised('train.txt', epoch=10)

In [ ]:
results = model.test('valid.txt')
precision, recall, examples = results
print(f'Test Precision: {precision}')
print(f'Test Recall: {recall}')

Test Precision: 492
Test Recall: 0.8109756097560976


In [ ]:
# Get the new review text
new_review = "This product is amazing, I highly recommend it."

In [ ]:
# Get the vector representation of the new review
new_review_vector = model.get_sentence_vector(new_review)

In [ ]:
# Convert the NumPy array to a space-separated string
new_review_vector_str = " ".join(map(str, new_review_vector))

In [ ]:
# Predict the star rating
predictions = model.predict(new_review_vector_str)

In [ ]:
# Output the predicted star rating
print(predictions)

(('__label__5.0',), array([1.00000989]))


In [ ]:
star_rating_map = {
    '__label__1.0': 1.0,
    '__label__2.0': 2.0,
    '__label__3.0': 3.0,
    '__label__4.0': 4.0,
    '__label__5.0': 5.0,
}

In [ ]:
# Get the predicted star rating as an integer
predicted_rating = star_rating_map[predictions[0][0]]

In [ ]:
# Print the predicted star rating
print(f'Predicted rating: {predicted_rating} stars')

Predicted rating: 5.0 stars
